In [ ]:
# Imports 
import numpy as np 
import matplotlib.pyplot as plt
import keras
from keras import layers
import io 
import imageio
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import tensorflow as tf

%matplotlib inline


In [ ]:
fpath = "val_dataset_128.npy"
dataset = np.load(fpath)

In [ ]:
def plot_arr(arr, hide_axis=False, norm = False):
    if norm:
        # Normalize the array values to the range [-1, 1]
        arr = arr / np.max(np.abs(arr))
    # Create a custom colormap that goes from blue (negative) to white (zero) to red (positive)
    cmap = plt.cm.bwr
    v_min = np.min([np.min(arr),-1])
    v_max = np.max([np.max(arr),1])
    plt.imshow(arr, cmap=cmap, interpolation='nearest', vmin=v_min, vmax=v_max)
    if hide_axis:
        plt.axis('off')
    plt.colorbar()  # Add a colorbar to show the scale
    plt.show()

In [ ]:
def vec_norm(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm


In [ ]:
def chi_index(reference_dose, evaluated_dose, distance_criterion=3, dose_criterion=0.03, pixel_spacing=2.72):
    """
    Calculate the chi index between two 2D dose distributions using the Bakai method.

    Args:
        reference_dose (np.array): 2D numpy array of the reference dose distribution (shape: 128x128x1)
        evaluated_dose (np.array): 2D numpy array of the evaluated dose distribution (shape: 128x128x1)
        distance_criterion (int): Distance criterion (in mm)
        dose_criterion (float): Dose criterion (in the same unit as the dose distributions)
        pixel_spacing (float): Pixel spacing (in mm) representing the distance between pixels (default: 2.72)

    Returns:
        chi_map (np.array): 2D numpy array of chi indices
    """
    # Ensure the arrays have the same shape
    assert reference_dose.shape == evaluated_dose.shape, "Reference and evaluated dose arrays must have the same shape."
    assert reference_dose.shape[2] == 1 and evaluated_dose.shape[2] == 1, "Input arrays should have a shape of 128x128x1."

    # Squeeze the arrays to remove the extra dimension
    reference_dose = vec_norm(np.squeeze(reference_dose))
    evaluated_dose = vec_norm(np.squeeze(evaluated_dose))
    
    
    delta_r = (distance_criterion / pixel_spacing)
    if np.max(reference_dose)==0:
        return np.expand_dims(evaluated_dose/delta_r, axis=2)

    dose_difference = evaluated_dose - reference_dose

    # Calculate the gradient of the reference dose distribution
    gradient_y, gradient_x = np.gradient(reference_dose, pixel_spacing)
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    delta_r = (distance_criterion / pixel_spacing)


    chi_map = dose_difference / np.sqrt(   delta_r**2 + (dose_criterion**2) * gradient_magnitude)
    return np.expand_dims(chi_map, axis=-1)


In [ ]:
model_path = "finetuned_ConvLSTM_Model_128_dice_10.h5"


loaded_model = tf.keras.models.load_model(model_path, compile=False)
bce_loss = tf.keras.losses.BinaryCrossentropy()
optimizer = keras.optimizers.Adam()
loaded_model.compile(loss=bce_loss, optimizer=optimizer)

In [ ]:
import numpy as np

def threshold_array(arr, threshold=0.5):
    """
    Set array values between 0 and 1 to 0 or 1 based on a threshold.

    Parameters:
        arr (np.array): Input array.
        threshold (float): Threshold value between 0 and 1 to decide whether to set values to 0 or 1.

    Returns:
        np.array: Modified array where values between 0 and 1 are set to 0 if below threshold and 1 if above.
    """
    # Ensuring the threshold is within the valid range
    if not (0 <= threshold <= 1):
        raise ValueError("Threshold must be between 0 and 1")

    # Apply threshold to values between 0 and 1
    arr[(arr > 0) & (arr < 1)] = (arr[(arr > 0) & (arr < 1)] >= threshold).astype(int)
    return arr


In [ ]:
def mask_to_bbox(mask, dilation=0):
    three_dim=False
    
    if len(mask.shape)==3:
        three_dim = True
        mask = np.squeeze(mask)
    rows, cols = np.nonzero(mask)
    
    if len(rows) == 0 or len(cols) == 0:
        return mask

    min_row, max_row = np.min(rows), np.max(rows)
    min_col, max_col = np.min(cols), np.max(cols)

    min_row = max(0, min_row - dilation)
    max_row = min(mask.shape[0] - 1, max_row + dilation)
    min_col = max(0, min_col - dilation)
    max_col = min(mask.shape[1] - 1, max_col + dilation)

    bbox = np.zeros_like(mask)


    bbox[min_row:max_row+1, min_col:max_col+1] = 1
    
    return bbox

In [ ]:
import numpy as np
def ROI_passrate(D_R,D_E,ROI_type='union', dose_criterion=0.03):
    if ROI_type=='union':
        ROI = np.logical_or(D_R,D_E)
    elif ROI_type=='union_bb':
        ROI = np.expand_dims(mask_to_bbox(np.squeeze(np.logical_or(D_R,D_E)), dilation=int(0.005 * D_R.shape[0])),axis=-1)
    elif ROI_type=='whole':
        ROI = np.ones((128,128,1), dtype=bool)
    else:
        raise Exception("Unknown ROI type")
    ROI_chi_map = chi_index(D_R, D_E)[ROI]
    
    if ROI_chi_map.shape!=(0,):
        pass_rate = (np.abs(ROI_chi_map)<dose_criterion).sum() / ROI.sum()
        return pass_rate
    #both masks are black:
    return 1
    

In [ ]:
whole_frame_alg_pass = []
whole_frame_avg_pass = []
whole_frame_blk_pass = []

union_bb_alg_pass = []
union_bb_avg_pass = []
union_bb_blk_pass = []


union_only_alg_pass = []
union_only_avg_pass = []
union_only_blk_pass = []



for idx in range(len(dataset)):
    example = dataset[idx]

    frames = example[:19, ...]
    ground_truth_frame = example[19, ...]

    new_prediction = loaded_model.predict(np.expand_dims(frames, axis=0))
    new_prediction = np.squeeze(new_prediction, axis=0)
    predicted_20th_frame = new_prediction[-1, ...]

    cd = ground_truth_frame
    md = predicted_20th_frame
    count = (np.abs(chi_index(cd,md)) < 0.03).sum()


    avg_d = np.expand_dims(np.squeeze(np.mean(frames,axis=0)), axis=-1)
    blk_d = np.zeros(cd.shape)
    count_avg = (np.abs(chi_index(cd,avg_d)) < 0.03).sum()
    
    
    plt.figure(figsize=(20, 5))

    # Ground truth
    plt.subplot(1, 4, 1)
    plt.imshow(np.squeeze(ground_truth_frame), cmap="gray")
    plt.title(f"Ground Truth (Frame 20) of {idx}")
    plt.axis("off")

    # Predicted
    plt.subplot(1, 4, 2)
    plt.imshow(np.squeeze(predicted_20th_frame), cmap="gray")
    plt.title(f"Predi (Frame 20) of {idx}")
    plt.axis("off")
    
    
    # average
    plt.subplot(1, 4, 3)
    plt.imshow(np.squeeze(avg_d), cmap="gray")
    plt.title(f"Avg of 19 frames of {idx}")
    plt.axis("off")
    

    # black
    plt.subplot(1, 4, 4)
    plt.imshow(np.squeeze(blk_d), cmap="gray")
    plt.title(f"Black frame")
    plt.axis("off")

    
    plt.show()


    whole_frame_alg_pass.append(ROI_passrate(cd,md,ROI_type='whole'))
    whole_frame_avg_pass.append(ROI_passrate(cd,avg_d,ROI_type='whole'))
    whole_frame_blk_pass.append(ROI_passrate(cd,blk_d,ROI_type='whole'))

    union_bb_alg_pass.append(ROI_passrate(cd,md,ROI_type='union_bb'))
    union_bb_avg_pass.append(ROI_passrate(cd,avg_d,ROI_type='union_bb'))
    union_bb_blk_pass.append(ROI_passrate(cd,blk_d,ROI_type='union_bb'))


    union_only_alg_pass.append(ROI_passrate(cd,md,ROI_type='union'))
    union_only_avg_pass.append(ROI_passrate(cd,avg_d,ROI_type='union'))
    union_only_blk_pass.append(ROI_passrate(cd,blk_d,ROI_type='union'))


In [ ]:
print(np.mean(whole_frame_alg_pass))
print(np.mean(whole_frame_avg_pass))
print(np.mean(whole_frame_blk_pass))

print(np.mean(union_bb_alg_pass))
print(np.mean(union_bb_avg_pass))
print(np.mean(union_bb_blk_pass))


print(np.mean(union_only_alg_pass))
print(np.mean(union_only_avg_pass))
print(np.mean(union_only_blk_pass))
